In [ ]:
!pip install nb_black
%load_ext nb_black

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from statsmodels.nonparametric.smoothers_lowess import lowess
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv(
    "/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv",
    index_col="enrollee_id",
)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique() / len(df)

In [ ]:
obj_cols = df.select_dtypes("object").columns
df[obj_cols].nunique()

In [ ]:
for col in obj_cols:
    print(col, df[col].unique())

In [ ]:
df["enrolled_university"] = pd.Categorical(
    df["enrolled_university"],
    categories=["no_enrollment", "Part time course", "Full time course"],
    ordered=True,
)

In [ ]:
df["education_level"] = pd.Categorical(
    df["education_level"],
    categories=["Primary School", "High School", "Masters", "Graduate"],
    ordered=True,
)

In [ ]:
df["experience"] = pd.Categorical(
    df["experience"],
    categories=[
        "<1",
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "9",
        "10",
        "11",
        "12",
        "13",
        "14",
        "15",
        "16",
        "17",
        "18",
        "19",
        "20",
        ">20",
    ],
    ordered=True,
)

In [ ]:
df["company_size"] = pd.Categorical(
    df["company_size"],
    categories=[
        "<10",
        "10/49",
        "50-99",
        "100-500",
        "500-999",
        "1000-4999",
        "5000-9999",
        "10000+",
    ],
    ordered=True,
)

In [ ]:
df["last_new_job"] = pd.Categorical(
    df["last_new_job"], categories=["never", "1", "2", "3", "4", ">4"], ordered=True,
)

In [ ]:
obj_cols = df.select_dtypes("object").columns
df[obj_cols] = df[obj_cols].astype("category")

In [ ]:
df.info()

In [ ]:
y = df["target"]
X = df.drop("target", axis=1)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
Xt, Xv, yt, yv = train_test_split(X, y, random_state=0)
dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)

In [ ]:
import seaborn as sns

In [ ]:
def loguniform(low=0, high=1, size=None, base=10):
    return np.power(base, np.random.uniform(low, high, size))

In [ ]:
best_etas = {"eta": [], "score": []}

In [ ]:
metric = "binary_logloss"
for _ in range(30):
    eta = loguniform(-3, 0)
    best_etas["eta"].append(eta)
    model = lgb.train(
        {"objective": "binary", "eta": eta},
        dt,
        num_boost_round=10000,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    best_etas["score"].append(model.best_score["valid"][metric])

best_eta_df = pd.DataFrame.from_dict(best_etas)
lowess_data = lowess(best_eta_df["score"], best_eta_df["eta"],)

plt.xscale("log")
rounded_data = lowess_data.copy()
rounded_data[:, 1] = rounded_data[:, 1].round(4)
rounded_data = rounded_data[::-1]  # reverse to find first minimum
good_enough_eta = rounded_data[rounded_data[:, 1].argmin(), 0]

# plot relationship between learning rate and performance, with its corresponding optimal eta
print(f"Good learning rate: {good_enough_eta}")
plt.axvline(good_enough_eta, color="orange")
plt.title("Smoothed relationship between learning rate and cost.")
plt.xlabel("learning rate")
plt.ylabel(metric)
sns.lineplot(lowess_data[:, 0], lowess_data[:, 1])

In [ ]:
model = lgb.train(
    {"objective": "binary", "eta": good_enough_eta},
    dt,
    num_boost_round=10000,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    early_stopping_rounds=50,
    verbose_eval=100,
)

In [ ]:
lgb.plot_importance(model, grid=False)

In [ ]:
threshold = 0.75
corr = Xt.corr(method="kendall")
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
upper = upper.stack()
high_upper = upper[(abs(upper) > threshold)]
abs_high_upper = abs(high_upper).sort_values(ascending=False)
pairs = abs_high_upper.index.to_list()
print(f"Correlated features: {pairs if len(pairs) > 0 else None}")

In [ ]:
sorted_features = [
    feature
    for _, feature in sorted(
        zip(model.feature_importance(), dt.feature_name), reverse=False
    )
]

best_score = model.best_score["valid"][metric]
print(f"starting score: {best_score:.4f}")
drop_unimportant_features = []
for feature in sorted_features:
    drop_unimportant_features.append(feature)
    X_train, X_valid, y_train, y_valid = train_test_split(
        X.drop(drop_unimportant_features, axis=1), y, random_state=0
    )
    dt = lgb.Dataset(X_train, y_train)
    dv = lgb.Dataset(X_valid, y_valid)
    drop_model = lgb.train(
        {"objective": "binary", "metric": metric, "eta": good_enough_eta},
        dt,
        valid_sets=[dt, dv],
        valid_names=["training", "valid"],
        num_boost_round=10000,
        early_stopping_rounds=50,
        verbose_eval=False,
    )
    score = drop_model.best_score["valid"][metric]
    if score > best_score:
        del drop_unimportant_features[-1]  # remove from drop list
        print(f"Dropping {feature} worsened score to {score:.4f}.")
        break
    else:
        best_score = score
print(f"ending score: {best_score:.4f}")
print(
    f"dropped features: {drop_unimportant_features if len(drop_unimportant_features) > 0 else None}"
)

In [ ]:
X = X.drop(drop_unimportant_features, axis=1)
Xt, Xv, yt, yv = train_test_split(X, y, random_state=0)
dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)

In [ ]:
import optuna.integration.lightgbm as lgb

params = {
    "objective": "binary",
    "metric": metric,
    "verbosity": -1,
    "boosting_type": "gbdt",
    "eta": good_enough_eta,
}

dt = lgb.Dataset(Xt, yt)
dv = lgb.Dataset(Xv, yv)


model = lgb.train(
    params,
    dt,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    num_boost_round=10000,
    verbose_eval=False,
    early_stopping_rounds=50,
)

score = model.best_score["valid"][metric]

best_params = model.params
print("Best params:", best_params)
print(f"  {metric} = {score}")
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

In [ ]:
import lightgbm as lgb

In [ ]:
best_params["eta"] = good_enough_eta

In [ ]:
model = lgb.train(
    best_params,
    dt,
    num_boost_round=10000,
    valid_sets=[dt, dv],
    valid_names=["training", "valid"],
    early_stopping_rounds=50,
    verbose_eval=100,
)

In [ ]:
lgb.plot_importance(model, grid=False, figsize=(5, 10))

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
ypred = model.predict(Xv, num_iteration=model.best_iteration)
print(f"AUC: {roc_auc_score(yv, ypred):.4f}")

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Xv)

In [ ]:
from category_encoders import OrdinalEncoder

In [ ]:
Xv_enc = OrdinalEncoder().fit_transform(Xv)

In [ ]:
for name in reversed(sorted_features):
    shap.dependence_plot(name, shap_values[1], Xv_enc, display_features=Xv)

In [ ]:
def preprocess_test(df, drop_unimportant_features):
    df = df.copy()

    df["enrolled_university"] = pd.Categorical(
        df["enrolled_university"],
        categories=["no_enrollment", "Part time course", "Full time course"],
        ordered=True,
    )

    df["education_level"] = pd.Categorical(
        df["education_level"],
        categories=["Primary School", "High School", "Masters", "Graduate"],
        ordered=True,
    )

    df["experience"] = pd.Categorical(
        df["experience"],
        categories=[
            "<1",
            "1",
            "2",
            "3",
            "4",
            "5",
            "6",
            "7",
            "8",
            "9",
            "10",
            "11",
            "12",
            "13",
            "14",
            "15",
            "16",
            "17",
            "18",
            "19",
            "20",
            ">20",
        ],
        ordered=True,
    )

    df["company_size"] = pd.Categorical(
        df["company_size"],
        categories=[
            "<10",
            "10/49",
            "50-99",
            "100-500",
            "500-999",
            "1000-4999",
            "5000-9999",
            "10000+",
        ],
        ordered=True,
    )

    df["last_new_job"] = pd.Categorical(
        df["last_new_job"],
        categories=["never", "1", "2", "3", "4", ">4"],
        ordered=True,
    )

    obj_cols = df.select_dtypes("object").columns
    df[obj_cols] = df[obj_cols].astype("category")

    X = df.drop(drop_unimportant_features, axis=1)
    return X

In [ ]:
df_test = pd.read_csv(
    "/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv",
    index_col="enrollee_id",
)

In [ ]:
X_test = preprocess_test(df_test, drop_unimportant_features)

In [ ]:
ypred_test = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
submission = pd.Series(ypred_test, X_test.index)